In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, set_seed
import os
import pandas as pd
import numpy as np
from typing import List, Tuple, Dict, Any

seed = 18022004
np.random.seed(seed)
set_seed(seed)

/drive2/phatnt/zTrans/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_prefix: str = 'data'
repo_prefix: str = f'{data_prefix}/repos'

model_id: str = 'deepseek-ai/deepseek-coder-6.7b-instruct'

In [3]:
data_name = 'migration_others_method_no_code.parquet'

data_df: pd.DataFrame = pd.read_parquet(f'{data_prefix}/{data_name}', engine = 'pyarrow')

In [4]:
data_df

,id,fromLib,toLib,repoName,prevCommit,startCommit,endCommit,fileName,startCommitChanges,endCommitChanges,...,method_before_modifiers,method_after_modifiers,method_before_return_type,method_after_return_type,method_before_parameters,method_after_parameters,method_before_signature,method_after_signature,method_before_signature_no_mod,method_after_signature_no_mod
0,0,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-compiler/src/main/java/org/drools/rule/...,+org.mockito:mockito-all,-org.jmock:jmock\n-org.jmock:jmock-legacy,...,private,private,void,void,"final RuleBuildContext context, final String o...","final RuleBuildContext context, final String o...",private void rewriteModify(final RuleBuildCont...,private void rewriteModify(final RuleBuildCont...,void rewriteModify(final RuleBuildContext cont...,void rewriteModify(final RuleBuildContext cont...
1,1,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-compiler/src/test/java/org/drools/guvno...,+org.mockito:mockito-all,-org.jmock:jmock\n-org.jmock:jmock-legacy,...,public,public,void,void,,,public void testAddSentenceMultipleTypes(),public void testAddSentenceMultipleTypes(),void testAddSentenceMultipleTypes(),void testAddSentenceMultipleTypes()
2,2,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-compiler/src/test/java/org/drools/integ...,+org.mockito:mockito-all,-org.jmock:jmock\n-org.jmock:jmock-legacy,...,public,public,void,void,,,public void testDynamicRuleRemovals(),public void testDynamicRuleRemovals(),void testDynamicRuleRemovals(),void testDynamicRuleRemovals()
3,3,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-compiler/src/test/java/org/drools/integ...,+org.mockito:mockito-all,-org.jmock:jmock\n-org.jmock:jmock-legacy,...,public,public,void,void,,,public void testLockOnActiveWithModify2(),public void testLockOnActiveWithModify2(),void testLockOnActiveWithModify2(),void testLockOnActiveWithModify2()
4,4,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-compiler/src/test/java/org/drools/integ...,+org.mockito:mockito-all,-org.jmock:jmock\n-org.jmock:jmock-legacy,...,public,public,void,void,,,public void testUpdateActivationCreationNoLoop(),public void testUpdateActivationCreationNoLoop(),void testUpdateActivationCreationNoLoop(),void testUpdateActivationCreationNoLoop()
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171774,171774,ru.yandex.qatools.properties:properties-loader,ru.qatools.commons:properties,yandex-qatools_hamcrest-pojo-matcher-generator,13ac1e25cc715b6855890070099a3cdbea36cf56,cc7a474d3ea53164d28b4a1b23c25989982e7d03,cc7a474d3ea53164d28b4a1b23c25989982e7d03,feature-matcher-generator/src/test/java/ru/yan...,+ru.qatools.commons:properties\n-ru.yandex.qat...,+ru.qatools.commons:properties\n-ru.yandex.qat...,...,@Test public,@Test public,void,void,,,@Test public void shouldCollectFieldsOfTwoClas...,@Test public void shouldCollectFieldsOfTwoClas...,void shouldCollectFieldsOfTwoClassesSeparated(),void shouldCollectFieldsOfTwoClassesSeparated()
171775,171775,ru.yandex.qatools.properties:properties-loader,ru.qatools.commons:properties,yandex-qatools_hamcrest-pojo-matcher-generator,13ac1e25cc715b6855890070099a3cdbea36cf56,cc7a474d3ea53164d28b4a1b23c25989982e7d03,cc7a474d3ea53164d28b4a1b23c25989982e7d03,feature-matcher-generator/src/test/java/ru/yan...,+ru.qatools.commons:properties\n-ru.yandex.qat.

In [6]:
# remove all special cases
# special cases are neither methods_before nor methods_after is blank
filtered_df = data_df[(data_df['method_before'].str.len() > 0) & (data_df['method_after'].str.len() > 0)]
special_case_df = data_df[(data_df['method_before'].str.len() == 0) | (data_df['method_after'].str.len() == 0)]

In [7]:
filtered_df

,id,fromLib,toLib,repoName,repoOwner,repoSplitName,prevCommit,startCommit,endCommit,fileName,...,method_before_modifiers,method_after_modifiers,method_before_return_type,method_after_return_type,method_before_parameters,method_after_parameters,method_before_signature,method_after_signature,method_before_signature_no_mod,method_after_signature_no_mod
0,0,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,bobmcwhirter,drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-compiler/src/main/java/org/drools/rule/...,...,private,private,void,void,"final RuleBuildContext context, final String o...","final RuleBuildContext context, final String o...",private void rewriteModify(final RuleBuildCont...,private void rewriteModify(final RuleBuildCont...,void rewriteModify(final RuleBuildContext cont...,void rewriteModify(final RuleBuildContext cont...
1,1,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,bobmcwhirter,drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-compiler/src/test/java/org/drools/guvno...,...,public,public,void,void,,,public void testAddSentenceMultipleTypes(),public void testAddSentenceMultipleTypes(),void testAddSentenceMultipleTypes(),void testAddSentenceMultipleTypes()
2,2,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,bobmcwhirter,drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-compiler/src/test/java/org/drools/integ...,...,public,public,void,void,,,public void testDynamicRuleRemovals(),public void testDynamicRuleRemovals(),void testDynamicRuleRemovals(),void testDynamicRuleRemovals()
3,3,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,bobmcwhirter,drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-compiler/src/test/java/org/drools/integ...,...,public,public,void,void,,,public void testUpdateActivationCreationNoLoop(),public void testUpdateActivationCreationNoLoop(),void testUpdateActivationCreationNoLoop(),void testUpdateActivationCreationNoLoop()
4,4,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,bobmcwhirter,drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-compiler/src/test/java/org/drools/integ...,...,public,public,void,void,,,public void testLockOnActiveWithModify2(),public void testLockOnActiveWithModify2(),void testLockOnActiveWithModify2(),void testLockOnActiveWithModify2()
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154546,154546,ru.yandex.qatools.properties:properties-loader,ru.qatools.commons:properties,yandex-qatools_hamcrest-pojo-matcher-generator,yandex-qatools,hamcrest-pojo-matcher-generator,13ac1e25cc715b6855890070099a3cdbea36cf56,cc7a474d3ea53164d28b4a1b23c25989982e7d03,cc7a474d3ea53164d28b4a1b23c25989982e7d03,feature-matcher-generator/src/test/java/ru/yan...,...,@Test public,@Test public,void,void,,,@Test public void shouldCollectFieldsOfTwoClas...,@Test public void shouldCollectFieldsOfTwoClas...,void shouldCollectFieldsOfTwoClassesSeparated(),void shouldCollectFieldsOfTwoClassesSeparated()
154547,154547,ru.yandex.qatools.properties:properties-loader,ru.qatools.commons:properties,yandex-qatools_hamcrest-pojo-matcher-generator,yandex-qatools,hamcrest-pojo-matcher-generator,13ac1e25cc715b6855890070099a3cdbea36cf56,cc7a474d3ea53164d28b4a1b23c25989982e7d03,cc7a474d3ea53164d28b4a1b23c25989982e7d03,feature-matcher-generator/src/test/java/ru/yan...,...,@Test public,@Test public,void,void,,,@Test public void shouldAskPackAndClassNames(),@Test public void shouldAskPackAndClassNames(),void shouldAskPackAndClassNames(),void shouldAskPackAndClassNames()
154548,154548,ru.yandex.qatools.properties:properties-loader,ru.qatools.commons:properties,yande

In [8]:
# check if row with the same 'fileName' have same 'total_methods_before' and 'total_methods_after'
# Check if all rows with the same 'fileName' and 'startCommit' have the same 'total_methods_before' and 'total_methods_after'
inconsistent_groups = filtered_df.groupby(['fileName', 'startCommit', 'endCommit']).apply(
    lambda x: not (
        x['total_methods_before'].nunique() == 1 and
        x['total_methods_after'].nunique() == 1
    )
)

# Filter inconsistent groups
inconsistent_pairs = inconsistent_groups[inconsistent_groups].index.tolist()

if not inconsistent_pairs:
    print("All groups have matching 'total_methods_before' and 'total_methods_after'.")
else:
    print(len(inconsistent_pairs), "inconsistent groups found.")
    print("Inconsistent groups:", inconsistent_pairs)

All groups have matching 'total_methods_before' and 'total_methods_after'.


/tmp/ipykernel_883809/1724684390.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  inconsistent_groups = filtered_df.groupby(['fileName', 'startCommit', 'endCommit']).apply(


BY MIGRATION

In [9]:
unique_commit_method_mapping = (
    filtered_df.groupby(['startCommit', 'endCommit'])
    .agg(repoName=('repoName', 'first'), count=('repoName', 'size'))
    .reset_index()
    .sort_values(by='count', ascending=False)
)

unique_commit_method_mapping

,startCommit,endCommit,repoName,count
426,5b6305c462e769f6af7c5692ea2f6b187ad80780,f9c9074d89edd036d8c18caebdb4a85a040513d7,openl-tablets_openl-tablets,7158
859,d59c7d6f4485bdd578ee5c45dfc421a705d48bc0,2a903b90e3a5fb015d59ccef4ccdde265b1746d8,mulesoft_mule,5364
561,7d603c302dccccfa80d8572a2825142431908eb0,191354e0378b4dd2257a6368ba7159348a651409,owlcs_owlapi,3437
851,d28d8235630d3a5d2940598a8fd562039a078c57,a881c12aae720a817d1cadc393902d2ba583c762,apache_groovy,3016
370,4f6732a7e9ac5df843990d6141955d71a5f633bd,d85662bd6e2a56bbbb8f753fede2036dfeba740f,eclipse_jetty.project,2879
...,...,...,...,...
38,09ac1f93712640f32b80e9809a392d5229d5bbb3,09ac1f93712640f32b80e9809a392d5229d5bbb3,puppetlabs_geppetto,1
53,0cf9677745345db47298c18f2f0e874e58dba9fe,0cf9677745345db47298c18f2f0e874e58dba9fe,ning_Arecibo,1
54,0d05070b6c4f03d232648844023eab4c8ebfb8ae,0d05070b6c4f03d232648844023eab4c8ebfb8ae,cjbi_wetech-admin,1
10,0339ab121d7d42eb0aa986c607c9dd236f42aa9a,0339ab121d7d42eb0aa986c607c9dd236f42aa9a,apache_metamodel,1


In [10]:
unique_commit_method_mapping.describe()

,count
count,1014.000000
mean,72.174556
std,375.703544
min,1.000000
25%,3.000000
50%,8.000000
75%,26.000000
max,7158.000000


In [11]:
# Group by 'startCommit' and 'endCommit' and calculate the required aggregations
unique_commit_method_mapping = (
    filtered_df.groupby(['startCommit', 'endCommit'])
    .agg(
        repoName=('repoName', 'first'),
        count=('repoName', 'size'),  # The number of rows (method change count)
        total_methods_before_sum=('total_methods_before', 'sum'),
        total_methods_after_sum=('total_methods_after', 'sum')
    )
    .reset_index()
    .sort_values(by='count', ascending=False)
)

# Calculate the percentage of method changes based on the 'count' (number of changes)
unique_commit_method_mapping['method_change_percentage_before'] = (
    (unique_commit_method_mapping['count'] / unique_commit_method_mapping['total_methods_before_sum']) * 100
)

unique_commit_method_mapping['method_change_percentage_after'] = (
    (unique_commit_method_mapping['count'] / unique_commit_method_mapping['total_methods_after_sum']) * 100
)

# Display the result with relevant columns
result = unique_commit_method_mapping[['startCommit', 'endCommit', 'repoName', 'count', 'method_change_percentage_before', 'method_change_percentage_after']]

result

,startCommit,endCommit,repoName,count,method_change_percentage_before,method_change_percentage_after
426,5b6305c462e769f6af7c5692ea2f6b187ad80780,f9c9074d89edd036d8c18caebdb4a85a040513d7,openl-tablets_openl-tablets,7158,1.344799,1.529768
859,d59c7d6f4485bdd578ee5c45dfc421a705d48bc0,2a903b90e3a5fb015d59ccef4ccdde265b1746d8,mulesoft_mule,5364,6.325546,5.790593
561,7d603c302dccccfa80d8572a2825142431908eb0,191354e0378b4dd2257a6368ba7159348a651409,owlcs_owlapi,3437,1.250965,1.098062
851,d28d8235630d3a5d2940598a8fd562039a078c57,a881c12aae720a817d1cadc393902d2ba583c762,apache_groovy,3016,1.273896,1.077782
370,4f6732a7e9ac5df843990d6141955d71a5f633bd,d85662bd6e2a56bbbb8f753fede2036dfeba740f,eclipse_jetty.project,2879,3.670321,3.328362
...,...,...,...,...,...,...
38,09ac1f93712640f32b80e9809a392d5229d5bbb3,09ac1f93712640f32b80e9809a392d5229d5bbb3,puppetlabs_geppetto,1,4.545455,4.545455
53,0cf9677745345db47298c18f2f0e874e58dba9fe,0cf9677745345db47298c18f2f0e874e58dba9fe,ning_Arecibo,1,50.000000,50.000000
54,0d05070b6c4f03d232648844023eab4c8ebfb8ae,0d05070b6c4f03d232648844023eab4c8ebfb8ae,cjbi_wetech-admin,1,7.692308,7.692308
10,0339ab121d7d42eb0aa986c607c9dd236f42aa9a,0339ab121d7d42eb0aa986c607c9dd236f42aa9a,apache_metamodel,1,16.666667,16.666667


In [12]:
unique_commit_method_mapping.describe()

,count,total_methods_before_sum,total_methods_after_sum,method_change_percentage_before,method_change_percentage_after
count,1014.000000,1.014000e+03,1.014000e+03,1014.000000,1014.000000
mean,72.174556,6.454365e+03,6.582615e+03,14.359484,14.637697
std,375.703544,1.198738e+05,1.206455e+05,17.236699,18.220189
min,1.000000,1.000000e+00,1.000000e+00,0.040135,0.039860
25%,3.000000,2.100000e+01,2.100000e+01,5.004181,4.901961
50%,8.000000,9.300000e+01,9.100000e+01,8.662370,8.544669
75%,26.000000,4.265000e+02,4.427500e+02,16.666667,16.666667
max,7158.000000,3.754802e+06,3.780770e+06,100.000000,100.000000


In [13]:
# Group by 'startCommit' and 'endCommit' and count the number of unique 'fileName' entries in each group
unique_commit_class_mapping = (
    filtered_df.groupby(['startCommit', 'endCommit'])
    .agg(repoName=('repoName', 'first'), file_count=('fileName', 'nunique'))
    .reset_index()
    .sort_values(by='file_count', ascending=False)
)

unique_commit_class_mapping

,startCommit,endCommit,repoName,file_count
859,d59c7d6f4485bdd578ee5c45dfc421a705d48bc0,2a903b90e3a5fb015d59ccef4ccdde265b1746d8,mulesoft_mule,1630
426,5b6305c462e769f6af7c5692ea2f6b187ad80780,f9c9074d89edd036d8c18caebdb4a85a040513d7,openl-tablets_openl-tablets,1479
370,4f6732a7e9ac5df843990d6141955d71a5f633bd,d85662bd6e2a56bbbb8f753fede2036dfeba740f,eclipse_jetty.project,707
561,7d603c302dccccfa80d8572a2825142431908eb0,191354e0378b4dd2257a6368ba7159348a651409,owlcs_owlapi,581
866,d68640c5a7995fbdbe0aa61dc88b0f0b23b6d5fe,3cfa7b24ee562abfabdf19bc1035c61c3ed3eff2,ProgrammeVitam_vitam,521
...,...,...,...,...
52,0cd7343c847dfd77549be001c079b7b97dfd1c1e,0cd7343c847dfd77549be001c079b7b97dfd1c1e,glowroot_glowroot,1
53,0cf9677745345db47298c18f2f0e874e58dba9fe,0cf9677745345db47298c18f2f0e874e58dba9fe,ning_Arecibo,1
54,0d05070b6c4f03d232648844023eab4c8ebfb8ae,0d05070b6c4f03d232648844023eab4c8ebfb8ae,cjbi_wetech-admin,1
57,0d9a7d0b2ecda371290e9b2635b5e81dda347bd5,0d9a7d0b2ecda371290e9b2635b5e81dda347bd5,apache_manifoldcf,1


In [14]:
unique_commit_class_mapping.describe()

,file_count
count,1014.000000
mean,20.023669
std,87.927489
min,1.000000
25%,2.000000
50%,4.000000
75%,11.000000
max,1630.000000


In [15]:
# Group by 'fileName' and calculate the required aggregations
unique_file_method_mapping = (
    filtered_df.groupby(['fileName'])
    .agg(
        repoName=('repoName', 'first'),
        count=('repoName', 'size'),  # The number of rows (method change count)
        total_methods_before_sum=('total_methods_before', 'sum'),
        total_methods_after_sum=('total_methods_after', 'sum')
    )
    .reset_index()
    .sort_values(by='count', ascending=False)
)

# Calculate the percentage of method changes based on the 'count' (number of changes)
unique_file_method_mapping['method_change_percentage_before'] = (
    (unique_file_method_mapping['count'] / unique_file_method_mapping['total_methods_before_sum']) * 100
)

unique_file_method_mapping['method_change_percentage_after'] = (
    (unique_file_method_mapping['count'] / unique_file_method_mapping['total_methods_after_sum']) * 100
)

# Display the result with relevant columns
result = unique_file_method_mapping[['fileName', 'repoName', 'count', 'method_change_percentage_before', 'method_change_percentage_after']]

result


,fileName,repoName,count,method_change_percentage_before,method_change_percentage_after
346,DEV/org.openl.grammars/src/org/openl/grammar/b...,openl-tablets_openl-tablets,332,0.177936,0.294118
9578,metastore/src/gen/thrift/gen-javabean/org/apac...,apache_hive,246,0.006738,0.006701
1048,DEV/org.openl.rules/test/org/openl/rules/helpe...,openl-tablets_openl-tablets,242,0.129534,0.121803
15988,src/main/org/codehaus/groovy/classgen/AsmClass...,apache_groovy,236,0.653595,0.512821
16377,src/test/org/codehaus/groovy/antlr/treewalker/...,apache_groovy,220,0.558659,0.564972
...,...,...,...,...,...
15,01-spring/01-spring-simple/src/test/java/com/m...,zccodere_study-imooc,1,100.000000,100.000000
16,01-spring/01-spring-simple/src/test/java/com/m...,zccodere_study-imooc,1,100.000000,100.000000
17,02-spring/02-spring-tx/src/main/java/com/myimo...,zccodere_study-imooc,1,25.000000,25.000000
18,02-spring/02-spring-tx/src/main/java/com/myimo...,zccodere_study-imooc,1,100.000000,100.000000


HANDLE SPECIAL CASES

In [16]:
special_case_df

,id,fromLib,toLib,repoName,repoOwner,repoSplitName,prevCommit,startCommit,endCommit,fileName,...,method_before_modifiers,method_after_modifiers,method_before_return_type,method_after_return_type,method_before_parameters,method_after_parameters,method_before_signature,method_after_signature,method_before_signature_no_mod,method_after_signature_no_mod
14,14,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,bobmcwhirter,drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-compiler/src/test/java/org/drools/reteo...,...,,public,,void,,"InternalFactHandle factHandle, ModifyPreviousT...",,public void modifyLeftTuple(InternalFactHandle...,,void modifyLeftTuple(InternalFactHandle factHa...
15,15,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,bobmcwhirter,drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-compiler/src/test/java/org/drools/reteo...,...,,public,,void,,"LeftTuple leftTuple, PropagationContext contex...",,public void modifyLeftTuple(LeftTuple leftTupl...,,"void modifyLeftTuple(LeftTuple leftTuple, Prop..."
16,16,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,bobmcwhirter,drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-compiler/src/test/java/org/drools/reteo...,...,,public,,void,,"RightTuple rightTuple, PropagationContext cont...",,public void modifyRightTuple(RightTuple rightT...,,"void modifyRightTuple(RightTuple rightTuple, P..."
22,22,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,bobmcwhirter,drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-core/src/main/java/org/drools/WorkingMe...,...,public,,void,,"final org.drools.FactHandle factHandle, final ...",,public void modifyInsert(final org.drools.Fact...,,void modifyInsert(final org.drools.FactHandle ...,
23,23,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,bobmcwhirter,drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-core/src/main/java/org/drools/WorkingMe...,...,public,,void,,final org.drools.FactHandle factHandle,,public void modifyRetract(final org.drools.Fac...,,void modifyRetract(final org.drools.FactHandle...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154533,154533,ru.yandex.qatools.properties:properties-loader,ru.qatools.commons:properties,yandex-qatools_hamcrest-pojo-matcher-generator,yandex-qatools,hamcrest-pojo-matcher-generator,13ac1e25cc715b6855890070099a3cdbea36cf56,cc7a474d3ea53164d28b4a1b23c25989982e7d03,cc7a474d3ea53164d28b4a1b23c25989982e7d03,feature-matcher-generator/src/main/java/ru/yan...,...,,@Override public,,void,,ClassDescription from,,@Override public void accept(ClassDescription ...,,void accept(ClassDescription from)
154534,154534,ru.yandex.qatools.properties:properties-loader,ru.qatools.commons:properties,yandex-qatools_hamcrest-pojo-matcher-generator,yandex-qatools,hamcrest-pojo-matcher-generator,13ac1e25cc715b6855890070099a3cdbea36cf56,cc7a474d3ea53164d28b4a1b23c25989982e7d03,cc7a474d3ea53164d28b4a1b23c25989982e7d03,feature-matcher-generator/src/main/java/ru/yan...,...,@Override public,,Void,,Element elem,,@Override public Void convert(Element elem),,Void convert(Element elem),
154535,154535,ru.yandex.qatools.properties:properties-loader,ru.qatools.commons:properties,yandex-qatools_hamcrest-pojo-matcher-generator,yandex-qatools,hamcrest-pojo-matcher-generator,13ac1e25cc715b6855890070099a3cdbea36cf56,cc7a474d3ea53164d28b4a1b23c25989982e7d03,cc7a474d3ea53164d28b4a1b23c25989982e7d03,feature-matcher-generator/src/main/java/ru/yan...,...,,@Override public,,void,,Element elem,,@Override public void accept(Element elem),,void accept(Element elem)
154538,154538,ru.yandex.qatools.pr

In [17]:
columns_to_check = special_case_df.columns.difference(['id'])

# Identify duplicate rows (excluding 'id' column)
duplicate_rows = special_case_df[special_case_df.duplicated(subset=columns_to_check, keep=False)]

# Group duplicates by their non-id columns
duplicates_grouped = duplicate_rows.groupby(list(columns_to_check))

# Display results
for values, group in duplicates_grouped:
    print(f"Duplicate Values (excluding 'id'): {values}")
    print(group, "\n")

In [18]:

# Extract rows based on `id`
row1 = special_case_df[special_case_df['id'] == 129401].iloc[0]
row2 = special_case_df[special_case_df['id'] == 129408].iloc[0]

# Compare rows and find differences
comparison = row1 != row2
differences = row1[comparison].to_frame(name='Row 30268').join(
    row2[comparison].to_frame(name='Row 105335')
)

print("Detailed comparison of differences:")
print(differences)


Detailed comparison of differences:
                                       Row 30268  \
id                                        129401   
class_after  DataValidationReducer.FileOffsetKey   

                                                 Row 105335  
id                                                   129408  
class_after  DataValidationReducer.ViolationPersistenceBean  


In [19]:
df_a = special_case_df[special_case_df["method_before_signature_no_mod"] != ""]
df_b = special_case_df[special_case_df["method_after_signature_no_mod"] != ""]

# Perform the merge as before
same_method_name_df = pd.merge(
    df_a, 
    df_b, 
    left_on="method_before_signature_no_mod", 
    right_on="method_after_signature_no_mod", 
    suffixes=("_a", "_b")
)

# Select the required columns, including id_a, id_b, and other necessary columns
same_method_name_df = same_method_name_df[[
    "id_a", "id_b",
    "repoName_a", "repoName_b",
    "class_before_a", "class_after_b",
    "fileName_a", 
    "startCommit_a", 
    "endCommit_a", 
    "method_before_a", 
    "fileName_b", 
    "startCommit_b", 
    "endCommit_b", 
    "method_after_b"
]]

# Rename columns for better readability
same_method_name_df.rename(columns={
    "id_a": "id_before",
    "id_b": "id_after",
    "repoName_a": "repoName_before",
    "repoName_b": "repoName_after",
    "class_before_a": "class_before",
    "class_after_b": "class_after",
    "fileName_a": "fileName_before",
    "startCommit_a": "startCommit_before",
    "endCommit_a": "endCommit_before",
    "method_before_a": "method_before",
    "fileName_b": "fileName_after",
    "startCommit_b": "startCommit_after",
    "endCommit_b": "endCommit_after",
    "method_after_b": "method_after",
}, inplace=True)

# Output the result
same_method_name_df

,id_before,id_after,repoName_before,repoName_after,class_before,class_after,fileName_before,startCommit_before,endCommit_before,method_before,fileName_after,startCommit_after,endCommit_after,method_after
0,461,92,bobmcwhirter_drools,bobmcwhirter_drools,ReteDslTestEngine.DslStep,ScheduledAgendaItem,drools-core/src/test/java/org/drools/reteoo/te...,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,public String toString() {\n return...,drools-core/src/main/java/org/drools/common/Sc...,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,"public String toString() {\n return ""[S..."
1,461,1131,bobmcwhirter_drools,mulesoft_mule,ReteDslTestEngine.DslStep,NestedInvocationHandler,drools-core/src/test/java/org/drools/reteoo/te...,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,public String toString() {\n return...,core/src/main/java/org/mule/routing/nested/Nes...,cac0f385ca2eb51b91b2be5a515890e9839fe443,b8cb8c4aa26d3ccbdf1a95b8421877c29beb6b39,public String toString()\n {\n final...
2,461,1273,bobmcwhirter_drools,mulesoft_mule,ReteDslTestEngine.DslStep,XaTransaction,drools-core/src/test/java/org/drools/reteoo/te...,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,public String toString() {\n return...,core/src/main/java/org/mule/transaction/XaTran...,cac0f385ca2eb51b91b2be5a515890e9839fe443,b8cb8c4aa26d3ccbdf1a95b8421877c29beb6b39,public String toString()\n {\n retur...
3,461,5705,bobmcwhirter_drools,mulesoft_mule,ReteDslTestEngine.DslStep,CreditProfile,drools-core/src/test/java/org/drools/reteoo/te...,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,public String toString() {\n return...,examples/loanbroker-simple/src/main/java/org/m...,d59c7d6f4485bdd578ee5c45dfc421a705d48bc0,2a903b90e3a5fb015d59ccef4ccdde265b1746d8,@Override\n public String toString()\n {...
4,461,14722,bobmcwhirter_drools,openl-tablets_openl-tablets,ReteDslTestEngine.DslStep,DomainOpenClass,drools-core/src/test/java/org/drools/reteoo/te...,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,public String toString() {\n return...,DEV/org.openl.core/src/org/openl/types/impl/Do...,5b6305c462e769f6af7c5692ea2f6b187ad80780,f9c9074d89edd036d8c18caebdb4a85a040513d7,@Override\n public String toString() {\n ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64877,154437,140941,jprante_elasticsearch-jdbc,eclipse_jetty.project,MockRiverMouth,IteratingCallback,src/test/java/org/xbib/elasticsearch/river/jdb...,e13884c0142ffe5e4f755fb6c427e0ca895a9bde,e13884c0142ffe5e4f755fb6c427e0ca895a9bde,@Override\n public void close() {\n },jetty-util/src/main/java/org/eclipse/jetty/uti...,4f6732a7e9ac5df843990d6141955d71a5f633bd,d85662bd6e2a56bbbb8f753fede2036dfeba740f,public void close()\n {\n loop: whil...
64878,154437,144912,jprante_elasticsearch-jdbc,ModeShape_modeshape,MockRiverMouth,,src/test/java/org/xbib/elasticsearch/river/jdb...,e13884c0142ffe5e4f755fb6c427e0ca895a9bde,e13884c0142ffe5e4f755fb6c427e0ca895a9bde,@Override\n public void close() {\n },modeshape-jcr-api/src/main/java/org/modeshape/...,2d79c0eb00e37f5ec35651936fdb66ab5ba095a5,a87749a6f89d97fe166c2152646c680214bf11f3,@Override\n public void close();
64879,154437,145931,jprante_elasticsearch-jdbc,ModeShape_modeshape,MockRiverMouth,JcrQueryResult,src/test/java/org/xbib/elasticsearch/river/jdb...,e13884c0142ffe5e4f755fb6c427e0ca895a9bde,e13884c0142ffe5e4f755fb6c427e0ca895a9bde,@Override\n public void close() {\n },modeshape-jcr/src/main/java/org/modeshape/jcr/...,2d79c0eb00e37f5ec35651936fdb66ab5ba095a5,a87749a6f89d97fe166c2152646c680214bf11f3,@Override\n public void close() {\n ...
64880,154437,147456,jprante_elasticsearch-jdbc,ModeShape_modeshape,MockRiverMouth,TestUtil,src/test/java/org/xbib/elasticsearch/river/jdb...,e13884c0142ffe5e4f755fb6c427e0ca895a9bde,e13884c0142ffe5e4f755fb6c427e0ca895a9bde,@Override\n public v

In [20]:
# filter out rows have that the pair (startCommit_before, endCommit_before) and (startCommit_after, endCommit_after) are the same

same_migration_df = same_method_name_df[
    ((same_method_name_df['startCommit_before'] == same_method_name_df['startCommit_after']) &
      (same_method_name_df['endCommit_before'] == same_method_name_df['endCommit_after'])) & 
      (same_method_name_df['repoName_before'] == same_method_name_df['repoName_after'])
]
same_migration_df

,id_before,id_after,repoName_before,repoName_after,class_before,class_after,fileName_before,startCommit_before,endCommit_before,method_before,fileName_after,startCommit_after,endCommit_after,method_after
0,461,92,bobmcwhirter_drools,bobmcwhirter_drools,ReteDslTestEngine.DslStep,ScheduledAgendaItem,drools-core/src/test/java/org/drools/reteoo/te...,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,public String toString() {\n return...,drools-core/src/main/java/org/drools/common/Sc...,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,"public String toString() {\n return ""[S..."
630,1663,1653,mulesoft_mule,mulesoft_mule,ManagementNamespaceHandlerTestCase,Log4jAgentTestCase,modules/management/src/test/java/org/mule/mana...,cac0f385ca2eb51b91b2be5a515890e9839fe443,b8cb8c4aa26d3ccbdf1a95b8421877c29beb6b39,protected void doTearDown() throws Exception\n...,modules/management/src/test/java/org/mule/mana...,cac0f385ca2eb51b91b2be5a515890e9839fe443,b8cb8c4aa26d3ccbdf1a95b8421877c29beb6b39,protected void doTearDown() throws Exception\n...
631,1663,1668,mulesoft_mule,mulesoft_mule,ManagementNamespaceHandlerTestCase,JmxSupportTestCase,modules/management/src/test/java/org/mule/mana...,cac0f385ca2eb51b91b2be5a515890e9839fe443,b8cb8c4aa26d3ccbdf1a95b8421877c29beb6b39,protected void doTearDown() throws Exception\n...,modules/management/src/test/java/org/mule/mana...,cac0f385ca2eb51b91b2be5a515890e9839fe443,b8cb8c4aa26d3ccbdf1a95b8421877c29beb6b39,protected void doTearDown() throws Exception\n...
638,1942,1864,mulesoft_mule,mulesoft_mule,EventMetaDataPropagationTestCase,FunctionalTestCase,tests/integration/src/test/java/org/mule/test/...,cac0f385ca2eb51b91b2be5a515890e9839fe443,b8cb8c4aa26d3ccbdf1a95b8421877c29beb6b39,protected ConfigurationBuilder getBuilder() th...,tests/functional/src/main/java/org/mule/tck/Fu...,cac0f385ca2eb51b91b2be5a515890e9839fe443,b8cb8c4aa26d3ccbdf1a95b8421877c29beb6b39,protected ConfigurationBuilder getBuilder() th...
639,1973,1864,mulesoft_mule,mulesoft_mule,MuleClientListenerTestCase,FunctionalTestCase,tests/integration/src/test/java/org/mule/test/...,cac0f385ca2eb51b91b2be5a515890e9839fe443,b8cb8c4aa26d3ccbdf1a95b8421877c29beb6b39,protected ConfigurationBuilder getBuilder() th...,tests/functional/src/main/java/org/mule/tck/Fu...,cac0f385ca2eb51b91b2be5a515890e9839fe443,b8cb8c4aa26d3ccbdf1a95b8421877c29beb6b39,protected ConfigurationBuilder getBuilder() th...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64611,152946,152935,olivergierke_spring-restbucks-training,olivergierke_spring-restbucks-training,PaymentProcessIntegrationTest.LinkWithRelMatcher,AbstractWebIntegrationTest.LinkWithRelMatcher,rest-2/src/test/java/org/springsource/restbuck...,912494a213062a7589e53f71fa8b1cfa0ef0a481,912494a213062a7589e53f71fa8b1cfa0ef0a481,@Override\n\t\tpublic void match(MvcResult res...,rest-2/src/test/java/org/springsource/restbuck...,912494a213062a7589e53f71fa8b1cfa0ef0a481,912494a213062a7589e53f71fa8b1cfa0ef0a481,@Override\n\t\tpublic void match(MvcResult res...
64612,152947,152938,olivergierke_spring-restbucks-training,olivergierke_spring-restbucks-training,PaymentProcessIntegrationTest,AbstractWebIntegrationTest,rest-2/src/test/java/org/springsource/restbuck...,912494a213062a7589e53f71fa8b1cfa0ef0a481,912494a213062a7589e53f71fa8b1cfa0ef0a481,private ResultMatcher linkWithRelIsNotPresent(...,rest-2/src/test/java/org/springsource/restbuck...,912494a213062a7589e53f71fa8b1cfa0ef0a481,912494a213062a7589e53f71fa8b1cfa0ef0a481,protected ResultMatcher linkWithRelIsNotPresen...
64613,152948,152937,olivergierke_spring-restbucks-training,olivergierke_spring-restbucks-training,PaymentProcessIntegrationTest,AbstractWebIntegrationTest,rest-2/src/test/java/org/springsource/restbuck...,912494a213062a7589e53f71fa8b1cfa0ef0a481,912494a213062a7589e53f71fa8b1cfa0ef0a481,private ResultMatcher linkWithRelIsPresent(fin...,rest-2/src/test/java/org/springsource/restbuck...

In [21]:
import difflib

def get_diff(string1, string2):
    # Normalize by removing leading/trailing whitespace and replacing tabs with spaces
    normalized1 = [line.strip().replace('\t', '') for line in string1.splitlines()]
    normalized2 = [line.strip().replace('\t', '') for line in string2.splitlines()]

    # Generate the diff
    diff = difflib.unified_diff(
        normalized1,
        normalized2,
        lineterm=''
    )
    return '\n'.join(diff)

In [22]:
same_migration_df["method_diff"] = same_migration_df.apply(lambda row: get_diff(row["method_before"], row["method_after"]), axis=1)

/tmp/ipykernel_883809/2214736524.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  same_migration_df["method_diff"] = same_migration_df.apply(lambda row: get_diff(row["method_before"], row["method_after"]), axis=1)


In [23]:
same_migration_df

,id_before,id_after,repoName_before,repoName_after,class_before,class_after,fileName_before,startCommit_before,endCommit_before,method_before,fileName_after,startCommit_after,endCommit_after,method_after,method_diff
0,461,92,bobmcwhirter_drools,bobmcwhirter_drools,ReteDslTestEngine.DslStep,ScheduledAgendaItem,drools-core/src/test/java/org/drools/reteoo/te...,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,public String toString() {\n return...,drools-core/src/main/java/org/drools/common/Sc...,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,"public String toString() {\n return ""[S...","--- \n+++ \n@@ -1,3 +1,3 @@\n public String to..."
630,1663,1653,mulesoft_mule,mulesoft_mule,ManagementNamespaceHandlerTestCase,Log4jAgentTestCase,modules/management/src/test/java/org/mule/mana...,cac0f385ca2eb51b91b2be5a515890e9839fe443,b8cb8c4aa26d3ccbdf1a95b8421877c29beb6b39,protected void doTearDown() throws Exception\n...,modules/management/src/test/java/org/mule/mana...,cac0f385ca2eb51b91b2be5a515890e9839fe443,b8cb8c4aa26d3ccbdf1a95b8421877c29beb6b39,protected void doTearDown() throws Exception\n...,"--- \n+++ \n@@ -1,6 +1,6 @@\n protected void d..."
631,1663,1668,mulesoft_mule,mulesoft_mule,ManagementNamespaceHandlerTestCase,JmxSupportTestCase,modules/management/src/test/java/org/mule/mana...,cac0f385ca2eb51b91b2be5a515890e9839fe443,b8cb8c4aa26d3ccbdf1a95b8421877c29beb6b39,protected void doTearDown() throws Exception\n...,modules/management/src/test/java/org/mule/mana...,cac0f385ca2eb51b91b2be5a515890e9839fe443,b8cb8c4aa26d3ccbdf1a95b8421877c29beb6b39,protected void doTearDown() throws Exception\n...,"--- \n+++ \n@@ -1,6 +1,6 @@\n protected void d..."
638,1942,1864,mulesoft_mule,mulesoft_mule,EventMetaDataPropagationTestCase,FunctionalTestCase,tests/integration/src/test/java/org/mule/test/...,cac0f385ca2eb51b91b2be5a515890e9839fe443,b8cb8c4aa26d3ccbdf1a95b8421877c29beb6b39,protected ConfigurationBuilder getBuilder() th...,tests/functional/src/main/java/org/mule/tck/Fu...,cac0f385ca2eb51b91b2be5a515890e9839fe443,b8cb8c4aa26d3ccbdf1a95b8421877c29beb6b39,protected ConfigurationBuilder getBuilder() th...,"--- \n+++ \n@@ -1,12 +1,4 @@\n protected Confi..."
639,1973,1864,mulesoft_mule,mulesoft_mule,MuleClientListenerTestCase,FunctionalTestCase,tests/integration/src/test/java/org/mule/test/...,cac0f385ca2eb51b91b2be5a515890e9839fe443,b8cb8c4aa26d3ccbdf1a95b8421877c29beb6b39,protected ConfigurationBuilder getBuilder() th...,tests/functional/src/main/java/org/mule/tck/Fu...,cac0f385ca2eb51b91b2be5a515890e9839fe443,b8cb8c4aa26d3ccbdf1a95b8421877c29beb6b39,protected ConfigurationBuilder getBuilder() th...,"--- \n+++ \n@@ -1,6 +1,4 @@\n protected Config..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64611,152946,152935,olivergierke_spring-restbucks-training,olivergierke_spring-restbucks-training,PaymentProcessIntegrationTest.LinkWithRelMatcher,AbstractWebIntegrationTest.LinkWithRelMatcher,rest-2/src/test/java/org/springsource/restbuck...,912494a213062a7589e53f71fa8b1cfa0ef0a481,912494a213062a7589e53f71fa8b1cfa0ef0a481,@Override\n\t\tpublic void match(MvcResult res...,rest-2/src/test/java/org/springsource/restbuck...,912494a213062a7589e53f71fa8b1cfa0ef0a481,912494a213062a7589e53f71fa8b1cfa0ef0a481,@Override\n\t\tpublic void match(MvcResult res...,"--- \n+++ \n@@ -1,6 +1,9 @@\n @Override\n publ..."
64612,152947,152938,olivergierke_spring-restbucks-training,olivergierke_spring-restbucks-training,PaymentProcessIntegrationTest,AbstractWebIntegrationTest,rest-2/src/test/java/org/springsource/restbuck...,912494a213062a7589e53f71fa8b1cfa0ef0a481,912494a213062a7589e53f71fa8b1cfa0ef0a481,private ResultMatcher linkWithRelIsNotPresent(...,rest-2/src/test/java/org/springsource/restbuck...,912494a213062a7589e53f71fa8b1cfa0ef0a481,912494a213062a7589e53f71fa8b1cfa0ef0a481,protected ResultMatcher linkWithRelIsNotPresen...,"--- \n+++ \n@@ -1,3 +1,3 @@\n-private ResultMa..."
64613,

In [24]:
same_migration_df[same_migration_df["fileName_before"] != same_migration_df["fileName_after"]]

,id_before,id_after,repoName_before,repoName_after,class_before,class_after,fileName_before,startCommit_before,endCommit_before,method_before,fileName_after,startCommit_after,endCommit_after,method_after,method_diff
0,461,92,bobmcwhirter_drools,bobmcwhirter_drools,ReteDslTestEngine.DslStep,ScheduledAgendaItem,drools-core/src/test/java/org/drools/reteoo/te...,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,public String toString() {\n return...,drools-core/src/main/java/org/drools/common/Sc...,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,"public String toString() {\n return ""[S...","--- \n+++ \n@@ -1,3 +1,3 @@\n public String to..."
630,1663,1653,mulesoft_mule,mulesoft_mule,ManagementNamespaceHandlerTestCase,Log4jAgentTestCase,modules/management/src/test/java/org/mule/mana...,cac0f385ca2eb51b91b2be5a515890e9839fe443,b8cb8c4aa26d3ccbdf1a95b8421877c29beb6b39,protected void doTearDown() throws Exception\n...,modules/management/src/test/java/org/mule/mana...,cac0f385ca2eb51b91b2be5a515890e9839fe443,b8cb8c4aa26d3ccbdf1a95b8421877c29beb6b39,protected void doTearDown() throws Exception\n...,"--- \n+++ \n@@ -1,6 +1,6 @@\n protected void d..."
631,1663,1668,mulesoft_mule,mulesoft_mule,ManagementNamespaceHandlerTestCase,JmxSupportTestCase,modules/management/src/test/java/org/mule/mana...,cac0f385ca2eb51b91b2be5a515890e9839fe443,b8cb8c4aa26d3ccbdf1a95b8421877c29beb6b39,protected void doTearDown() throws Exception\n...,modules/management/src/test/java/org/mule/mana...,cac0f385ca2eb51b91b2be5a515890e9839fe443,b8cb8c4aa26d3ccbdf1a95b8421877c29beb6b39,protected void doTearDown() throws Exception\n...,"--- \n+++ \n@@ -1,6 +1,6 @@\n protected void d..."
638,1942,1864,mulesoft_mule,mulesoft_mule,EventMetaDataPropagationTestCase,FunctionalTestCase,tests/integration/src/test/java/org/mule/test/...,cac0f385ca2eb51b91b2be5a515890e9839fe443,b8cb8c4aa26d3ccbdf1a95b8421877c29beb6b39,protected ConfigurationBuilder getBuilder() th...,tests/functional/src/main/java/org/mule/tck/Fu...,cac0f385ca2eb51b91b2be5a515890e9839fe443,b8cb8c4aa26d3ccbdf1a95b8421877c29beb6b39,protected ConfigurationBuilder getBuilder() th...,"--- \n+++ \n@@ -1,12 +1,4 @@\n protected Confi..."
639,1973,1864,mulesoft_mule,mulesoft_mule,MuleClientListenerTestCase,FunctionalTestCase,tests/integration/src/test/java/org/mule/test/...,cac0f385ca2eb51b91b2be5a515890e9839fe443,b8cb8c4aa26d3ccbdf1a95b8421877c29beb6b39,protected ConfigurationBuilder getBuilder() th...,tests/functional/src/main/java/org/mule/tck/Fu...,cac0f385ca2eb51b91b2be5a515890e9839fe443,b8cb8c4aa26d3ccbdf1a95b8421877c29beb6b39,protected ConfigurationBuilder getBuilder() th...,"--- \n+++ \n@@ -1,6 +1,4 @@\n protected Config..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64611,152946,152935,olivergierke_spring-restbucks-training,olivergierke_spring-restbucks-training,PaymentProcessIntegrationTest.LinkWithRelMatcher,AbstractWebIntegrationTest.LinkWithRelMatcher,rest-2/src/test/java/org/springsource/restbuck...,912494a213062a7589e53f71fa8b1cfa0ef0a481,912494a213062a7589e53f71fa8b1cfa0ef0a481,@Override\n\t\tpublic void match(MvcResult res...,rest-2/src/test/java/org/springsource/restbuck...,912494a213062a7589e53f71fa8b1cfa0ef0a481,912494a213062a7589e53f71fa8b1cfa0ef0a481,@Override\n\t\tpublic void match(MvcResult res...,"--- \n+++ \n@@ -1,6 +1,9 @@\n @Override\n publ..."
64612,152947,152938,olivergierke_spring-restbucks-training,olivergierke_spring-restbucks-training,PaymentProcessIntegrationTest,AbstractWebIntegrationTest,rest-2/src/test/java/org/springsource/restbuck...,912494a213062a7589e53f71fa8b1cfa0ef0a481,912494a213062a7589e53f71fa8b1cfa0ef0a481,private ResultMatcher linkWithRelIsNotPresent(...,rest-2/src/test/java/org/springsource/restbuck...,912494a213062a7589e53f71fa8b1cfa0ef0a481,912494a213062a7589e53f71fa8b1cfa0ef0a481,protected ResultMatcher linkWithRelIsNotPresen...,"--- \n+++ \n@@ -1,3 +1,3 @@\n-private ResultMa..."
64613,

In [25]:
# count the number of unique pair (id_before, id_after) in the same_migration_df
same_migration_df.groupby(['id_before', 'id_after']).size().reset_index(name='count').sort_values(by='count', ascending=False)

,id_before,id_after,count
20879,152952,152936,1
0,461,92,1
1,1663,1653,1
2,1663,1668,1
3,1942,1864,1
...,...,...,...
19,2506,2451,1
20,2514,2670,1
21,2514,2909,1
22,2514,5855,1


In [26]:
# check if fileName_before and fileName_after are in the same package, extract the package name from the file name

same_migration_df["package_before"] = same_migration_df["fileName_before"].str.extract(r'(.*)/.*')
same_migration_df["package_after"] = same_migration_df["fileName_after"].str.extract(r'(.*)/.*')

same_migration_df["same_package"] = same_migration_df["package_before"] == same_migration_df["package_after"]

same_migration_df

/tmp/ipykernel_883809/2527286596.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  same_migration_df["package_before"] = same_migration_df["fileName_before"].str.extract(r'(.*)/.*')
/tmp/ipykernel_883809/2527286596.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  same_migration_df["package_after"] = same_migration_df["fileName_after"].str.extract(r'(.*)/.*')
/tmp/ipykernel_883809/2527286596.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

,id_before,id_after,repoName_before,repoName_after,class_before,class_after,fileName_before,startCommit_before,endCommit_before,method_before,fileName_after,startCommit_after,endCommit_after,method_after,method_diff,package_before,package_after,same_package
0,461,92,bobmcwhirter_drools,bobmcwhirter_drools,ReteDslTestEngine.DslStep,ScheduledAgendaItem,drools-core/src/test/java/org/drools/reteoo/te...,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,public String toString() {\n return...,drools-core/src/main/java/org/drools/common/Sc...,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,"public String toString() {\n return ""[S...","--- \n+++ \n@@ -1,3 +1,3 @@\n public String to...",drools-core/src/test/java/org/drools/reteoo/test,drools-core/src/main/java/org/drools/common,False
630,1663,1653,mulesoft_mule,mulesoft_mule,ManagementNamespaceHandlerTestCase,Log4jAgentTestCase,modules/management/src/test/java/org/mule/mana...,cac0f385ca2eb51b91b2be5a515890e9839fe443,b8cb8c4aa26d3ccbdf1a95b8421877c29beb6b39,protected void doTearDown() throws Exception\n...,modules/management/src/test/java/org/mule/mana...,cac0f385ca2eb51b91b2be5a515890e9839fe443,b8cb8c4aa26d3ccbdf1a95b8421877c29beb6b39,protected void doTearDown() throws Exception\n...,"--- \n+++ \n@@ -1,6 +1,6 @@\n protected void d...",modules/management/src/test/java/org/mule/mana...,modules/management/src/test/java/org/mule/mana...,False
631,1663,1668,mulesoft_mule,mulesoft_mule,ManagementNamespaceHandlerTestCase,JmxSupportTestCase,modules/management/src/test/java/org/mule/mana...,cac0f385ca2eb51b91b2be5a515890e9839fe443,b8cb8c4aa26d3ccbdf1a95b8421877c29beb6b39,protected void doTearDown() throws Exception\n...,modules/management/src/test/java/org/mule/mana...,cac0f385ca2eb51b91b2be5a515890e9839fe443,b8cb8c4aa26d3ccbdf1a95b8421877c29beb6b39,protected void doTearDown() throws Exception\n...,"--- \n+++ \n@@ -1,6 +1,6 @@\n protected void d...",modules/management/src/test/java/org/mule/mana...,modules/management/src/test/java/org/mule/mana...,False
638,1942,1864,mulesoft_mule,mulesoft_mule,EventMetaDataPropagationTestCase,FunctionalTestCase,tests/integration/src/test/java/org/mule/test/...,cac0f385ca2eb51b91b2be5a515890e9839fe443,b8cb8c4aa26d3ccbdf1a95b8421877c29beb6b39,protected ConfigurationBuilder getBuilder() th...,tests/functional/src/main/java/org/mule/tck/Fu...,cac0f385ca2eb51b91b2be5a515890e9839fe443,b8cb8c4aa26d3ccbdf1a95b8421877c29beb6b39,protected ConfigurationBuilder getBuilder() th...,"--- \n+++ \n@@ -1,12 +1,4 @@\n protected Confi...",tests/integration/src/test/java/org/mule/test/...,tests/functional/src/main/java/org/mule/tck,False
639,1973,1864,mulesoft_mule,mulesoft_mule,MuleClientListenerTestCase,FunctionalTestCase,tests/integration/src/test/java/org/mule/test/...,cac0f385ca2eb51b91b2be5a515890e9839fe443,b8cb8c4aa26d3ccbdf1a95b8421877c29beb6b39,protected ConfigurationBuilder getBuilder() th...,tests/functional/src/main/java/org/mule/tck/Fu...,cac0f385ca2eb51b91b2be5a515890e9839fe443,b8cb8c4aa26d3ccbdf1a95b8421877c29beb6b39,protected ConfigurationBuilder getBuilder() th...,"--- \n+++ \n@@ -1,6 +1,4 @@\n protected Config...",tests/integration/src/test/java/org/mule/test/...,tests/functional/src/main/java/org/mule/tck,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64611,152946,152935,olivergierke_spring-restbucks-training,olivergierke_spring-restbucks-training,PaymentProcessIntegrationTest.LinkWithRelMatcher,AbstractWebIntegrationTest.LinkWithRelMatcher,rest-2/src/test/java/org/springsource/restbuck...,912494a213062a7589e53f71fa8b1cfa0ef0a481,912494a213062a7589e53f71fa8b1cfa0ef0a481,@Override\n\t\tpublic void match(MvcResult res...,rest-2/src/test/java/org/springsource/restbuck...,912494a213062a7589e53f71fa8b1cfa0ef0a481,912494a213062a7589e53f71fa8b1cfa0ef0a481,@Override\n\t\tpublic void match(MvcResult res...,"--- \n+++ \n@@ -1,6 +1,9 @@\n @Override\n publ...",rest-2/src/t

In [27]:
# check if how many rows have the same package
same_migration_df["same_package"].value_counts()

same_package
False    20157
True       723
Name: count, dtype: int64

In [28]:
# get the rows with same package
same_package_df = same_migration_df[same_migration_df["same_package"]]

same_package_df

,id_before,id_after,repoName_before,repoName_after,class_before,class_after,fileName_before,startCommit_before,endCommit_before,method_before,fileName_after,startCommit_after,endCommit_after,method_after,method_diff,package_before,package_after,same_package
1494,3149,3186,mulesoft_mule,mulesoft_mule,AbstractMessageProcessorOwner,ResponseMessageProcessorAdapter,core/src/main/java/org/mule/processor/Abstract...,d59c7d6f4485bdd578ee5c45dfc421a705d48bc0,2a903b90e3a5fb015d59ccef4ccdde265b1746d8,public void stop() throws MuleException\n {...,core/src/main/java/org/mule/processor/Response...,d59c7d6f4485bdd578ee5c45dfc421a705d48bc0,2a903b90e3a5fb015d59ccef4ccdde265b1746d8,public void stop() throws MuleException\n {...,"--- \n+++ \n@@ -1,12 +1,7 @@\n public void sto...",core/src/main/java/org/mule/processor,core/src/main/java/org/mule/processor,True
1495,3149,3219,mulesoft_mule,mulesoft_mule,AbstractMessageProcessorOwner,TransactionalInterceptingMessageProcessor,core/src/main/java/org/mule/processor/Abstract...,d59c7d6f4485bdd578ee5c45dfc421a705d48bc0,2a903b90e3a5fb015d59ccef4ccdde265b1746d8,public void stop() throws MuleException\n {...,core/src/main/java/org/mule/processor/Transact...,d59c7d6f4485bdd578ee5c45dfc421a705d48bc0,2a903b90e3a5fb015d59ccef4ccdde265b1746d8,@Override\n public void stop() throws MuleE...,"--- \n+++ \n@@ -1,12 +1,8 @@\n+@Override\n pub...",core/src/main/java/org/mule/processor,core/src/main/java/org/mule/processor,True
1519,3150,3189,mulesoft_mule,mulesoft_mule,AbstractMessageProcessorOwner,ResponseMessageProcessorAdapter,core/src/main/java/org/mule/processor/Abstract...,d59c7d6f4485bdd578ee5c45dfc421a705d48bc0,2a903b90e3a5fb015d59ccef4ccdde265b1746d8,public void start() throws MuleException\n ...,core/src/main/java/org/mule/processor/Response...,d59c7d6f4485bdd578ee5c45dfc421a705d48bc0,2a903b90e3a5fb015d59ccef4ccdde265b1746d8,public void start() throws MuleException\n ...,"--- \n+++ \n@@ -1,11 +1,7 @@\n public void sta...",core/src/main/java/org/mule/processor,core/src/main/java/org/mule/processor,True
1520,3150,3215,mulesoft_mule,mulesoft_mule,AbstractMessageProcessorOwner,TransactionalInterceptingMessageProcessor,core/src/main/java/org/mule/processor/Abstract...,d59c7d6f4485bdd578ee5c45dfc421a705d48bc0,2a903b90e3a5fb015d59ccef4ccdde265b1746d8,public void start() throws MuleException\n ...,core/src/main/java/org/mule/processor/Transact...,d59c7d6f4485bdd578ee5c45dfc421a705d48bc0,2a903b90e3a5fb015d59ccef4ccdde265b1746d8,@Override\n public void start() throws Mule...,"--- \n+++ \n@@ -1,11 +1,8 @@\n+@Override\n pub...",core/src/main/java/org/mule/processor,core/src/main/java/org/mule/processor,True
1541,3151,3188,mulesoft_mule,mulesoft_mule,AbstractMessageProcessorOwner,ResponseMessageProcessorAdapter,core/src/main/java/org/mule/processor/Abstract...,d59c7d6f4485bdd578ee5c45dfc421a705d48bc0,2a903b90e3a5fb015d59ccef4ccdde265b1746d8,public void initialise() throws Initialisation...,core/src/main/java/org/mule/processor/Response...,d59c7d6f4485bdd578ee5c45dfc421a705d48bc0,2a903b90e3a5fb015d59ccef4ccdde265b1746d8,public void initialise() throws Initialisation...,"--- \n+++ \n@@ -1,18 +1,15 @@\n public void in...",core/src/main/java/org/mule/processor,core/src/main/java/org/mule/processor,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62501,147028,146609,ModeShape_modeshape,ModeShape_modeshape,JcrSessionTest,ImportExportTest,modeshape-jcr/src/test/java/org/modeshape/jcr/...,2d79c0eb00e37f5ec35651936fdb66ab5ba095a5,a87749a6f89d97fe166c2152646c680214bf11f3,"@FixFor( ""MODE-2284"" )\n @Test\n public ...",modeshape-jcr/src/test/java/org/modeshape/jcr/...,2d79c0eb00e37f5ec35651936fdb66ab5ba095a5,a87749a6f89d97fe166c2152646c680214bf11f3,"@FixFor( ""MODE-2284"" )\n @Test\n public ...",,modeshape-jcr/src/test/java/org/modeshape/jcr,modeshape-jcr/src/test/java/org/modeshape/jcr,True
62968,148112,148068,ModeShape_modeshape,ModeShape_modeshape,,Console,web/modeshape-web-explorer/src/main/java/

In [29]:
test_df = same_migration_df[same_method_name_df['startCommit_before'] == 'd59c7d6f4485bdd578ee5c45dfc421a705d48bc0']

test_df = test_df[test_df['endCommit_before'] == '2a903b90e3a5fb015d59ccef4ccdde265b1746d8']

test_df = test_df[test_df['repoName_before'] == 'mulesoft_mule']

/tmp/ipykernel_883809/1663626710.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test_df = same_migration_df[same_method_name_df['startCommit_before'] == 'd59c7d6f4485bdd578ee5c45dfc421a705d48bc0']


In [30]:
test_df = test_df[test_df['method_diff'].str.len() > 0]

In [31]:
len(test_df)

600

In [32]:
unique_codes = test_df['method_before'].unique()

cases = []

for id in range(len(unique_codes)):
    code = unique_codes[id]
    if (len(code.split('\n')) == 1):
        continue
    
    samples = test_df[test_df['method_before'] == unique_codes[id]]
    
    for sample_id in range(len(samples)):
        if (len(samples.iloc[sample_id]['method_after'].split('\n')) == 1):
            continue
        
        cases.append(
            {
                'method_before': samples.iloc[sample_id]['method_before'],
                'method_after': samples.iloc[sample_id]['method_after'],
            },
        )

for id in range(len(cases)):
    print(f"Case {id + 1}")
    print(f"Method before: {cases[id]['method_before']}")
    print(f"Method after: {cases[id]['method_after']}")
    print('-' * 50)
    print('\n')
    
    # if (id == 2):
    #     break

print(f"Total {len(cases)} cases")

Case 1
Method before: public EndpointURI getEndpointURI()
    {
        return event.getEndpoint().getEndpointURI();
    }
Method after: public EndpointURI getEndpointURI()
    {
        return null;
    }
--------------------------------------------------


Case 2
Method before: public void initAfterDeserialisation(MuleContext muleContext) throws MuleException
    {
        this.muleContext = muleContext;
    }
Method after: @Override
    public void initAfterDeserialisation(MuleContext muleContext) throws MuleException
    {
        super.initAfterDeserialisation(muleContext);
        this.toJmsMessage = new ObjectToJMSMessage();
        this.jmsConnector = (JmsConnector) this.connector;
    }
--------------------------------------------------


Case 3
Method before: public boolean isPersistent()
    {
        return persistent;
    }
Method after: @Override
    public boolean isPersistent()
    {
        return false;
    }
--------------------------------------------------


Case 4

In [33]:
# count number of unique id_before in the same_package_df

# same_package_df.groupby('id_before').size().reset_index(name='count').sort_values(by='count', ascending=False)

In [34]:
# count number of unique id_after in the same_package_df

# same_package_df.groupby('id_after').size().reset_index(name='count').sort_values(by='count', ascending=False)

In [35]:
# get row with diff == ""
move_methods = same_migration_df[same_migration_df["method_diff"] == ""]

move_methods

,id_before,id_after,repoName_before,repoName_after,class_before,class_after,fileName_before,startCommit_before,endCommit_before,method_before,fileName_after,startCommit_after,endCommit_after,method_after,method_diff,package_before,package_after,same_package
731,2506,2451,mulesoft_mule,mulesoft_mule,,,core/src/main/java/org/mule/api/config/MuleCon...,d59c7d6f4485bdd578ee5c45dfc421a705d48bc0,2a903b90e3a5fb015d59ccef4ccdde265b1746d8,String getClusterId();,core/src/main/java/org/mule/api/MuleContext.java,d59c7d6f4485bdd578ee5c45dfc421a705d48bc0,2a903b90e3a5fb015d59ccef4ccdde265b1746d8,String getClusterId();,,core/src/main/java/org/mule/api/config,core/src/main/java/org/mule/api,False
1030,2688,2207,mulesoft_mule,mulesoft_mule,DefaultMuleConfiguration,DefaultMuleContext,core/src/main/java/org/mule/config/DefaultMule...,d59c7d6f4485bdd578ee5c45dfc421a705d48bc0,2a903b90e3a5fb015d59ccef4ccdde265b1746d8,public String getClusterId()\n {\n r...,core/src/main/java/org/mule/DefaultMuleContext...,d59c7d6f4485bdd578ee5c45dfc421a705d48bc0,2a903b90e3a5fb015d59ccef4ccdde265b1746d8,public String getClusterId()\n {\n r...,,core/src/main/java/org/mule/config,core/src/main/java/org/mule,False
1547,3152,3187,mulesoft_mule,mulesoft_mule,AbstractMessageProcessorOwner,ResponseMessageProcessorAdapter,core/src/main/java/org/mule/processor/Abstract...,d59c7d6f4485bdd578ee5c45dfc421a705d48bc0,2a903b90e3a5fb015d59ccef4ccdde265b1746d8,public void setFlowConstruct(FlowConstruct flo...,core/src/main/java/org/mule/processor/Response...,d59c7d6f4485bdd578ee5c45dfc421a705d48bc0,2a903b90e3a5fb015d59ccef4ccdde265b1746d8,public void setFlowConstruct(FlowConstruct flo...,,core/src/main/java/org/mule/processor,core/src/main/java/org/mule/processor,True
1769,3463,3470,mulesoft_mule,mulesoft_mule,AbstractAuthenticationFilter,AbstractEndpointSecurityFilter,core/src/main/java/org/mule/security/AbstractA...,d59c7d6f4485bdd578ee5c45dfc421a705d48bc0,2a903b90e3a5fb015d59ccef4ccdde265b1746d8,protected abstract void authenticateInbound(Mu...,core/src/main/java/org/mule/security/AbstractE...,d59c7d6f4485bdd578ee5c45dfc421a705d48bc0,2a903b90e3a5fb015d59ccef4ccdde265b1746d8,protected abstract void authenticateInbound(Mu...,,core/src/main/java/org/mule/security,core/src/main/java/org/mule/security,True
1770,3465,3468,mulesoft_mule,mulesoft_mule,AbstractAuthenticationFilter,AbstractEndpointSecurityFilter,core/src/main/java/org/mule/security/AbstractA...,d59c7d6f4485bdd578ee5c45dfc421a705d48bc0,2a903b90e3a5fb015d59ccef4ccdde265b1746d8,protected abstract void authenticateOutbound(M...,core/src/main/java/org/mule/security/AbstractE...,d59c7d6f4485bdd578ee5c45dfc421a705d48bc0,2a903b90e3a5fb015d59ccef4ccdde265b1746d8,protected abstract void authenticateOutbound(M...,,core/src/main/java/org/mule/security,core/src/main/java/org/mule/security,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62501,147028,146609,ModeShape_modeshape,ModeShape_modeshape,JcrSessionTest,ImportExportTest,modeshape-jcr/src/test/java/org/modeshape/jcr/...,2d79c0eb00e37f5ec35651936fdb66ab5ba095a5,a87749a6f89d97fe166c2152646c680214bf11f3,"@FixFor( ""MODE-2284"" )\n @Test\n public ...",modeshape-jcr/src/test/java/org/modeshape/jcr/...,2d79c0eb00e37f5ec35651936fdb66ab5ba095a5,a87749a6f89d97fe166c2152646c680214bf11f3,"@FixFor( ""MODE-2284"" )\n @Test\n public ...",,modeshape-jcr/src/test/java/org/modeshape/jcr,modeshape-jcr/src/test/java/org/modeshape/jcr,True
64253,151019,151033,timurstrekalov_saga,timurstrekalov_saga,DefaultCoverageGenerator,InstanceFieldPerPropertyConfig,saga-core/src/main/java/com/github/timurstreka...,d7a2fdda43a079075902cb6ce4c2730a5207a2e1,d7a2fdda43a079075902cb6ce4c2730a5207a2e1,@Override\n public Pattern apply(fi...,saga-core/src/main/java/com/github/timurstreka...,d7a2fdda43a079075902cb6ce4c2730a5207a2e1,d7a2fdda43a079075902cb6ce4c2730a5207a2e1,@Override\n public Pattern apply(fi...,,saga-core/src/main/java/com/github/timurstreka...,saga-core/src/main/java/com/gi

In [36]:
move_methods.groupby('id_before').size().reset_index(name='count').sort_values(by='count', ascending=False)

,id_before,count
117,84544,4
10,24075,3
6,13603,3
12,24140,3
178,140605,3
...,...,...
182,147028,1
183,151019,1
184,151536,1
185,151587,1


In [37]:
move_methods.groupby('id_after').size().reset_index(name='count').sort_values(by='count', ascending=False)

,id_after,count
28,33642,31
30,33756,31
23,33181,7
22,33134,6
128,132782,3
...,...,...
146,141970,1
148,145161,1
149,146609,1
150,151033,1


In [38]:
move_methods[move_methods["id_before"] == 129358]

,id_before,id_after,repoName_before,repoName_after,class_before,class_after,fileName_before,startCommit_before,endCommit_before,method_before,fileName_after,startCommit_after,endCommit_after,method_after,method_diff,package_before,package_after,same_package
54849,129358,129401,Impetus_jumbune,Impetus_jumbune,DataDiscrepanciesArrayWritable,DataValidationReducer.FileOffsetKey,datavalidation/src/main/java/org/jumbune/datav...,6a93f641febd5a86ff6620a16552a9606ac08616,6a93f641febd5a86ff6620a16552a9606ac08616,public String getFileName() {\n\t\treturn file...,datavalidation/src/main/java/org/jumbune/datav...,6a93f641febd5a86ff6620a16552a9606ac08616,6a93f641febd5a86ff6620a16552a9606ac08616,public String getFileName() {\n\t\t\treturn fi...,,datavalidation/src/main/java/org/jumbune/datav...,datavalidation/src/main/java/org/jumbune/datav...,True
54850,129358,129408,Impetus_jumbune,Impetus_jumbune,DataDiscrepanciesArrayWritable,DataValidationReducer.ViolationPersistenceBean,datavalidation/src/main/java/org/jumbune/datav...,6a93f641febd5a86ff6620a16552a9606ac08616,6a93f641febd5a86ff6620a16552a9606ac08616,public String getFileName() {\n\t\treturn file...,datavalidation/src/main/java/org/jumbune/datav...,6a93f641febd5a86ff6620a16552a9606ac08616,6a93f641febd5a86ff6620a16552a9606ac08616,public String getFileName() {\n\t\t\treturn fi...,,datavalidation/src/main/java/org/jumbune/datav...,datavalidation/src/main/java/org/jumbune/datav...,True


In [39]:
move_methods[move_methods["id_after"] == 33134]

,id_before,id_after,repoName_before,repoName_after,class_before,class_after,fileName_before,startCommit_before,endCommit_before,method_before,fileName_after,startCommit_after,endCommit_after,method_after,method_diff,package_before,package_after,same_package
12332,33119,33134,apache_synapse,apache_synapse,AddressEndpointFactory,EndpointFactory,java/modules/core/src/main/java/org/apache/syn...,30d7a6a6620dfbf2baa902a1406e712f782f9efe,25656c9dfbd2e434dc205c5dcad6cbdfff462495,public Object getObjectFromOMNode(OMNode om) {...,java/modules/core/src/main/java/org/apache/syn...,30d7a6a6620dfbf2baa902a1406e712f782f9efe,25656c9dfbd2e434dc205c5dcad6cbdfff462495,public Object getObjectFromOMNode(OMNode om) {...,,java/modules/core/src/main/java/org/apache/syn...,java/modules/core/src/main/java/org/apache/syn...,True
12349,33146,33134,apache_synapse,apache_synapse,FailoverEndpointFactory,EndpointFactory,java/modules/core/src/main/java/org/apache/syn...,30d7a6a6620dfbf2baa902a1406e712f782f9efe,25656c9dfbd2e434dc205c5dcad6cbdfff462495,public Object getObjectFromOMNode(OMNode om) {...,java/modules/core/src/main/java/org/apache/syn...,30d7a6a6620dfbf2baa902a1406e712f782f9efe,25656c9dfbd2e434dc205c5dcad6cbdfff462495,public Object getObjectFromOMNode(OMNode om) {...,,java/modules/core/src/main/java/org/apache/syn...,java/modules/core/src/main/java/org/apache/syn...,True
12362,33152,33134,apache_synapse,apache_synapse,IndirectEndpointFactory,EndpointFactory,java/modules/core/src/main/java/org/apache/syn...,30d7a6a6620dfbf2baa902a1406e712f782f9efe,25656c9dfbd2e434dc205c5dcad6cbdfff462495,public Object getObjectFromOMNode(OMNode om) {...,java/modules/core/src/main/java/org/apache/syn...,30d7a6a6620dfbf2baa902a1406e712f782f9efe,25656c9dfbd2e434dc205c5dcad6cbdfff462495,public Object getObjectFromOMNode(OMNode om) {...,,java/modules/core/src/main/java/org/apache/syn...,java/modules/core/src/main/java/org/apache/syn...,True
12371,33158,33134,apache_synapse,apache_synapse,LoadbalanceEndpointFactory,EndpointFactory,java/modules/core/src/main/java/org/apache/syn...,30d7a6a6620dfbf2baa902a1406e712f782f9efe,25656c9dfbd2e434dc205c5dcad6cbdfff462495,public Object getObjectFromOMNode(OMNode om) {...,java/modules/core/src/main/java/org/apache/syn...,30d7a6a6620dfbf2baa902a1406e712f782f9efe,25656c9dfbd2e434dc205c5dcad6cbdfff462495,public Object getObjectFromOMNode(OMNode om) {...,,java/modules/core/src/main/java/org/apache/syn...,java/modules/core/src/main/java/org/apache/syn...,True
12380,33164,33134,apache_synapse,apache_synapse,SALoadbalanceEndpointFactory,EndpointFactory,java/modules/core/src/main/java/org/apache/syn...,30d7a6a6620dfbf2baa902a1406e712f782f9efe,25656c9dfbd2e434dc205c5dcad6cbdfff462495,public Object getObjectFromOMNode(OMNode om) {...,java/modules/core/src/main/java/org/apache/syn...,30d7a6a6620dfbf2baa902a1406e712f782f9efe,25656c9dfbd2e434dc205c5dcad6cbdfff462495,public Object getObjectFromOMNode(OMNode om) {...,,java/modules/core/src/main/java/org/apache/syn...,java/modules/core/src/main/java/org/apache/syn...,True
12393,33171,33134,apache_synapse,apache_synapse,WSDLEndpointFactory,EndpointFactory,java/modules/core/src/main/java/org/apache/syn...,30d7a6a6620dfbf2baa902a1406e712f782f9efe,25656c9dfbd2e434dc205c5dcad6cbdfff462495,public Object getObjectFromOMNode(OMNode om) {...,java/modules/core/src/main/java/org/apache/syn...,30d7a6a6620dfbf2baa902a1406e712f782f9efe,25656c9dfbd2e434dc205c5dcad6cbdfff462495,public Object getObjectFromOMNode(OMNode om) {...,,java/modules/core/src/main/java/org/apache/syn...,java/modules/core/src/main/java/org/apache/syn...,True
